In [ ]:
import pandas as pd
from keras.models import load_model

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/deepLearning/project/top_10_tags_encoded.csv', encoding='latin1')

In [ ]:
model = load_model("/content/drive/MyDrive/Colab Notebooks/deepLearning/project/lstm_multilabel_model.keras")

/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 8 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


I am loading the saved model that I traind in the past phase

In [ ]:
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

In [ ]:
# Parameters
MAX_NUM_WORDS = 10000
MAX_SEQUENCE_LENGTH = 300

# Tokenize
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(df['clean_text'])
sequences = tokenizer.texts_to_sequences(df['clean_text'])

# Pad
X_seq = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

# Labels (already one-hot encoded)
y = df.iloc[:, 1:].values  # assuming clean_text is the first column

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X_seq, y, test_size=0.2, random_state=42)

In [ ]:
# Predict probabilities on validation set
y_pred_probs = model.predict(X_val)

4411/4411 ━━━━━━━━━━━━━━━━━━━━ 363s 82ms/step


In [ ]:
print(y_pred_probs.shape);
print(y_pred_probs[: 5])

(141143, 10)
[[1.3742083e-01 8.0447520e-05 4.2072592e-05 2.9579774e-04 2.2695679e-04
  9.9088919e-01 3.2090952e-04 4.5822024e-05 9.0044586e-04 4.2599757e-04]
 [2.3912480e-03 2.5329317e-03 1.7092051e-04 4.7491857e-01 2.5283507e-04
  8.7853996e-03 9.4237328e-01 1.9064820e-01 4.4764105e-02 5.5983348e-04]
 [9.9907351e-01 8.6471031e-04 1.5171616e-04 1.0077984e-03 4.2597804e-04
  7.4797377e-02 7.0538226e-04 4.0577419e-04 7.1070957e-05 9.5933952e-05]
 [4.2406041e-03 1.9286251e-04 6.4736395e-04 7.1064476e-04 2.0546702e-05
  9.9795806e-01 2.6034662e-03 8.8846442e-05 9.8128425e-05 2.9508787e-04]
 [2.5365818e-05 8.6190802e-01 1.0530833e-02 1.2335751e-03 9.8581484e-04
  1.0896993e-02 8.8235073e-02 5.2736686e-03 9.8816614e-05 2.0790767e-02]]


Above is probabilities of the 10 classes for each test row



---

# Threshold tuning

In [ ]:
import numpy as np
from sklearn.metrics import f1_score

In [ ]:
best_thresholds = []
f1_scores = []

# Try multiple thresholds between 0.1 and 0.9
thresholds = np.arange(0.1, 0.91, 0.05)

for i in range(y_val.shape[1]):  # for each tag/class
    best_f1 = 0
    best_thresh = 0.5
    for thresh in thresholds:
        pred = (y_pred_probs[:, i] >= thresh).astype(int)
        score = f1_score(y_val[:, i], pred)
        if score > best_f1:
            best_f1 = score
            best_thresh = thresh
    best_thresholds.append(best_thresh)
    f1_scores.append(best_f1)

# Report
for i, (t, f1) in enumerate(zip(best_thresholds, f1_scores)):
    print(f"Class {i}: Best Threshold = {t:.2f}, F1 Score = {f1:.4f}")

Class 0: Best Threshold = 0.55, F1 Score = 0.9521
Class 1: Best Threshold = 0.45, F1 Score = 0.9006
Class 2: Best Threshold = 0.40, F1 Score = 0.8807
Class 3: Best Threshold = 0.35, F1 Score = 0.6546
Class 4: Best Threshold = 0.55, F1 Score = 0.9403
Class 5: Best Threshold = 0.55, F1 Score = 0.8727
Class 6: Best Threshold = 0.45, F1 Score = 0.8136
Class 7: Best Threshold = 0.50, F1 Score = 0.8208
Class 8: Best Threshold = 0.45, F1 Score = 0.9245
Class 9: Best Threshold = 0.60, F1 Score = 0.9549


-- copy of the output --

Class 0: Best Threshold = 0.55, F1 Score = 0.9521 </br>
Class 1: Best Threshold = 0.45, F1 Score = 0.9006 </br>
Class 2: Best Threshold = 0.40, F1 Score = 0.8807 </br>
Class 3: Best Threshold = 0.35, F1 Score = 0.6546 </br>
Class 4: Best Threshold = 0.55, F1 Score = 0.9403 </br>
Class 5: Best Threshold = 0.55, F1 Score = 0.8727 </br>
Class 6: Best Threshold = 0.45, F1 Score = 0.8136 </br>
Class 7: Best Threshold = 0.50, F1 Score = 0.8208 </br>
Class 8: Best Threshold = 0.45, F1 Score = 0.9245 </br>
Class 9: Best Threshold = 0.60, F1 Score = 0.9549 </br>

We now have the best threshold for each class. eg: Class 0 (Tag[0]) performs best when the threshold is 0.55, similarly Class 2 at 0.45

A model's prediction would be like: y_pred_probs = [0.4, 0.7, 0.6, 0.2, 0.9, 0.5, 0.3, 0.55, 0.65, 0.62], with a threshold of 0.5 flat value we will select everything above the threshold as 1 and hence it get classified as positive. The issue is you can’t just use a default 0.5 cutoff for every tag — each tag might perform better with a different threshold. Hence in the above code we have tuned and found optimal thresholds for eac class. So we will apply these thresholds individually for each class so the performance is now much better

In [ ]:
best_thresholds # has the best threshold for each class

[np.float64(0.5500000000000002),
 np.float64(0.45000000000000007),
 np.float64(0.40000000000000013),
 np.float64(0.3500000000000001),
 np.float64(0.5500000000000002),
 np.float64(0.5500000000000002),
 np.float64(0.45000000000000007),
 np.float64(0.5000000000000001),
 np.float64(0.45000000000000007),
 np.float64(0.6000000000000002)]

In [ ]:
# Final binary predictions using best thresholds
y_pred_binary = np.zeros_like(y_pred_probs) #create a np array with the same shape as y_pred_probs but filled entirely with 0

for i, thresh in enumerate(best_thresholds):
    y_pred_binary[:, i] = (y_pred_probs[:, i] >= thresh).astype(int)

In [ ]:
y_pred_binary

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

^ Now I am applying the thresholds to the validation. This will give me the final binary predictions using my optimized thresholds